In [1]:
import sys
import os

sys.path.append(os.path.abspath(os.path.join('..')))

In [2]:
import ee
import geemap

from vegetation_period_NDVI.time_series import extract_time_ranges, get_harmonic_ts
from vegetation_period_NDVI.vegetation_period_extraction import (
    get_crop_veg_period,
    create_binary_ndvi_indicator,
    create_binary_mask,
)
from typing import List

*** Earth Engine *** Share your feedback by taking our Annual Developer Satisfaction Survey: https://google.qualtrics.com/jfe/form/SV_0JLhFqfSY1uiEaW?source=Init


In [3]:
ee.Initialize(project="ee-sahellakes")


In [7]:
double_cropping = ee.Image("projects/thurgau-irrigation/assets/Zuerich/crop_vegetation_period_zh_2022")

double_cropping.bandNames().getInfo()

['firstStart', 'firstEnd', 'secondStart', 'secondEnd', 'isDoubleCropping']

In [8]:
Map = geemap.Map()

vis_params = {
    "bands": ["isDoubleCropping"],
    "min": 0,
    "max": 1,
    "palette": ["white", "green"],
}
Map.addLayer(double_cropping, vis_params, "Double Cropping")

Map

Map(center=[0, 0], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=SearchDataGUI(childr…

## Exporting the vegetation period images as an asset to the project

### Define the Area of Interest

In [4]:
# cantonal_borders_asset = "projects/thurgau-irrigation/assets/Zuerich/Zuerich_bound"

# aoi_feature_collection = ee.FeatureCollection(cantonal_borders_asset)
# aoi_geometry = aoi_feature_collection.geometry()
# aoi_geometry = aoi_geometry.simplify(500)
# aoi_buffered = aoi_geometry.buffer(100)

# aoi_name = "Zuerich"


# List of countries
countries = ee.List([
    'Albania',
    'Bulgaria',
    'Bosnia & Herzegovina',
    'Croatia',
    'Moldova',
    'Macedonia',
    'Montenegro',
    'Poland',
    'Romania',
    'Serbia',
    'Turkey',
    'Kosovo'
])

# Load the FeatureCollection
grid_results = ee.FeatureCollection('projects/ee-et-data/assets/ECA/GRID_area_irrg_WorldCereal_Corine_v2')
borders=ee.FeatureCollection("USDOS/LSIB_SIMPLE/2017")
# Map over the FeatureCollection to add a 'combined' property
grid_results = grid_results.map(lambda ft: ft.set(
    'combined',
    ee.Number(ft.get('Corine')).gt(0).Or(ee.Number(ft.get('WorldCereal')).gt(1000))
))

# Filter features where 'combined' equals 1
loaded_collections = grid_results.filter(ee.Filter.eq('combined', 1))

# Function to filter borders for each country and combine them
def add_to_list(str, previous):
    previous = ee.List(previous)
    return previous.add(borders.filter(ee.Filter.eq('country_na', ee.String(str))).first())

# Accumulate borders for the specified countries
borders_ECA = ee.List(countries.iterate(add_to_list, ee.List([])))
borders_ECA = ee.FeatureCollection(borders_ECA)

# Filter loadedCollections to the bounds of the borders_ECA
loaded_collections = loaded_collections.filterBounds(borders_ECA)

# Map over loadedCollections to set 'fid' and 'scheme_property_name'
scheme_property_name = 'fid_property'  # Replace with your actual scheme property name
aoi_buffered = loaded_collections.map(lambda ft: ft.set(
        'fid', ee.Number(ft.get('fid'))
    ).set(
        scheme_property_name, ee.String('fid_').cat(ee.String(ft.get('fid')))
    ))
fids=aoi_buffered.aggregate_array('fid').getInfo()
# print(len(fids))

# Get the first feature and its geometry
aoi_feature = aoi_buffered.filter(ee.Filter.eq('fid',27721)).first()
aoi_geometry = aoi_feature.geometry()

# Define the AOI
aoi = aoi_geometry

# Define a crop mask
WorldCereal=ee.ImageCollection("ESA/WorldCereal/2021/MODELS/v100");
CORINE=ee.Image("COPERNICUS/CORINE/V20/100m/2018")
CORINE_mosaic=CORINE.eq(212).Or(CORINE.eq(213)).selfMask()

# Define the LULC mask using WorldCereal and blend with CORINE mosaic
lulc = (
    WorldCereal.filter(ee.Filter.eq('product', 'temporarycrops')).max()
    .select('classification').gt(50).selfMask()
    .updateMask(
        WorldCereal.filter(ee.Filter.eq('product', 'temporarycrops')).max()
        .select('confidence').gt(50)
    )
    .unmask(0)
    .blend(CORINE_mosaic)
    .selfMask()
)

#keep only WorldCereal_temporarycrops, mask all the rest
#clip with country boundaries
#make sure the date of the WAPOR image is the middle of the month
#file name: use fid

### Define the years of interest

In [5]:
years = [2021]

### Export the Vegetation Periods images to the project assets

In [6]:
print(len(fids[432:]))
index = fids.index(27721)  # Get the index of 27300
print(index)  # Output: 2

58
4


In [ ]:
# fid_name=27721
for fid in fids[100:]:
    fid_name=fid
    # Filter the aoi_buffered collection for the current fid
    aoi_geometry = aoi_buffered.filter(ee.Filter.eq('fid', fid)).first().geometry()
    for year in years:

        print(f"Processing year {year}")

        time_intervals = extract_time_ranges([f"{year}-03-01", f"{year}-10-31"], 15)

        image_to_export = get_crop_veg_period(year, aoi_geometry, time_intervals)

        task_name = f"crop_veg_period_{year}_F{fid_name}"
        asset_id = f"projects/ee-et-data/assets/ECA/ET_data/VegPeriods/crop_veg_period_{year}_F{fid_name}"

        bands_to_export = [
            "firstStart",
            "firstEnd",
            "secondStart",
            "secondEnd",
            "isDoubleCropping",
        ]

        image_to_export = image_to_export.select(bands_to_export).updateMask(lulc.eq(1)).set("fid", fid_name)
        # print(image_to_export.getInfo())
        task = ee.batch.Export.image.toAsset(
            image=image_to_export,
            description=task_name,
            assetId=asset_id,
            region=aoi_geometry,
            scale=10,
            maxPixels=1e13,
        )

        task.start()

        print(f"Exporting {task_name} to {asset_id}")

Processing year 2021
Exporting crop_veg_period_2021_F32626 to projects/ee-et-data/assets/ECA/ET_data/VegPeriods/crop_veg_period_2021_F32626
Processing year 2021
Exporting crop_veg_period_2021_F32627 to projects/ee-et-data/assets/ECA/ET_data/VegPeriods/crop_veg_period_2021_F32627
Processing year 2021
Exporting crop_veg_period_2021_F32628 to projects/ee-et-data/assets/ECA/ET_data/VegPeriods/crop_veg_period_2021_F32628
Processing year 2021
Exporting crop_veg_period_2021_F32629 to projects/ee-et-data/assets/ECA/ET_data/VegPeriods/crop_veg_period_2021_F32629
Processing year 2021
Exporting crop_veg_period_2021_F32630 to projects/ee-et-data/assets/ECA/ET_data/VegPeriods/crop_veg_period_2021_F32630
Processing year 2021
Exporting crop_veg_period_2021_F32631 to projects/ee-et-data/assets/ECA/ET_data/VegPeriods/crop_veg_period_2021_F32631
Processing year 2021
Exporting crop_veg_period_2021_F32632 to projects/ee-et-data/assets/ECA/ET_data/VegPeriods/crop_veg_period_2021_F32632
Processing year 2021

In [ ]:
print(image_to_export.projection().getInfo())

In [ ]:
# veg_2019 = ee.Image("projects/thurgau-irrigation/assets/Thurgau/VegetationPeriod/crop_veg_period_2018")

# Map = geemap.Map()
# Map.centerObject(aoi_geometry, 10)
# options = {
#     "bands": ["isDoubleCropping"],
#     "min": 0,
#     "max": 1,
#     "palette": ["blue", "red"],
# }
# Map.addLayer(veg_2019, options, "Crop Veg Period 2019")

# Map

## General Code Testing. Ignore this part

In [5]:
print(f"Processing year 2021")

time_intervals = extract_time_ranges([f"2021-03-01", f"2021-10-31"], 15)

image_to_export = get_crop_veg_period(2021, aoi_geometry, time_intervals)
print(image_to_export.bandNames().getInfo())

Processing year 2021
['firstStart', 'firstEnd', 'secondStart', 'secondEnd', 'isDoubleCropping']


In [ ]:
Map = geemap.Map()
Map.centerObject(aoi_geometry, 10)
options = {
    "bands": ["isDoubleCropping"],
    "min": 0,
    "max": 1,
    "palette": ["blue", "red"],
}
Map.addLayer(image_to_export, options, "Crop Veg Period 2019")

Map